In [67]:
from pyrosetta import *
from pyrosetta.rosetta import *
from MiningMinima import *

# Developing subroutines to integrate over rigid body dofs

## Initialize stuff

In [107]:
seq1, seq2 = 'g', 'c'
pose = protocols.recces.pose_setup_turner(seq1, seq2)
n_res = len(seq1) + len(seq2)

# set fold tree using usual chi torsion atoms
ft = FoldTree(n_res)
ft.new_jump(1, n_res, n_res/2)
ft.set_jump_atoms(1, pose.residue(1).atom_name(pose.residue(1).chi_atoms(1)[4]),
    pose.residue(n_res).atom_name(pose.residue(n_res).chi_atoms(1)[4]))
pose.fold_tree(ft)

# set jump dofs in movemap
movemap = MoveMap()
movemap.set_jump(1, True)

# get scorefxn
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner_new')

In [108]:
pmm = PyMOLMover()
pmm.apply(pose)

## Set up multifunc and minimize

In [140]:
# taken from MiningMinima class
def set_up_multifunc(pose, movemap, scorefxn):
    # set up minimizer map
    min_map = core.optimization.MinimizerMap()
    min_map.setup(pose, movemap)

    start_score = scorefxn(pose)
    pose.energies().set_use_nblist(pose, min_map.domain_map(), True)
    multifunc = core.optimization.AtomTreeMultifunc(pose, min_map, scorefxn)
    scorefxn.setup_for_minimizing(pose, min_map)
    scorefxn.setup_for_derivatives(pose)

    min_map = min_map
    multifunc = multifunc
    
    return multifunc, min_map

In [141]:
multifunc, min_map = set_up_multifunc(pose, movemap, scorefxn)
min_dofs = Vector1([0.0]*min_map.nangles())
min_map.copy_dofs_from_pose(pose,min_dofs)
start_func_val = multifunc(min_dofs)
theta = Vector1(list(min_dofs))
dE_dtheta = Vector1(list(min_dofs))
multifunc.dfunc(theta,dE_dtheta)

In [168]:
min_options = core.optimization.MinimizerOptions(
    'lbfgs_armijo_nonmonotone', 1e-15, True, False, False)
min_options.nblist_auto_update(True)
minimizer = core.optimization.Minimizer(multifunc, min_options)

In [169]:
min_dofs = Vector1(list(theta))
minimizer.run(min_dofs)

core.optimization.LineMinimizer: [ ERROR ] Inaccurate G! step= 3.72529e-09 Deriv= -1.67008e-08 Finite Diff= 0.0140457


-13.371328465939765

In [170]:
min_map.copy_dofs_to_pose(pose, min_dofs)
pmm.apply(pose)

In [173]:
pmm.keep_history(True)
min_dofs = np.array(min_dofs)
new_dofs = np.array(min_dofs)
print new_dofs

[  7.1598544   16.0415942   17.90825238   1.35448238   0.58155587
  -1.64427993]


In [181]:
new_dofs[0] += 10
min_map.copy_dofs_to_pose(pose, Vector1(list(new_dofs)))
pmm.apply(pose)

In [183]:
print new_dofs
print min_dofs

[ 37.1598544   16.0415942   17.90825238   1.35448238   0.58155587
  -1.64427993]
[  7.1598544   16.0415942   17.90825238   1.35448238   0.58155587
  -1.64427993]
